In [ ]:
%cd
  
!git clone --quiet https://github.com/matterport/Mask_RCNN.git

/root


Install libraries:

In [ ]:
%cd ~/Mask_RCNN

!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

/root/Mask_RCNN
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 298 kB 5.0 MB/s 
     |████████████████████████████████| 154 kB 72.8 MB/s 
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
runn

In [ ]:
!pip show mask-rcnn

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


Given some loading errors of MRCNN model, we downgrade keras and tensorflow:

In [ ]:
!pip uninstall keras -y
!pip uninstall h5py -y

!pip install keras==2.0.8
!pip install h5py==2.10.0

Found existing installation: Keras 2.3.1
Uninstalling Keras-2.3.1:
  Successfully uninstalled Keras-2.3.1
Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Successfully uninstalled h5py-2.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)


Given a warning about boolean dtype of the images, we downgrade scikit-image:

In [ ]:
!pip uninstall scikit-image -y
!pip install scikit-image==0.16.2

Found existing installation: scikit-image 0.16.2
Uninstalling scikit-image-0.16.2:
  Successfully uninstalled scikit-image-0.16.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_image-0.16.2-cp37-cp37m-manylinux1_x86_64.whl (26.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Load dataset:

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

TensorFlow 1.x selected.
Found GPU at: /device:GPU:0
Fri Jul  8 15:26:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    28W /  70W |    104MiB / 15109MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Building DatasetProcessing class, to have a compatibile dataset for RCNN:

In [ ]:
import numpy as np
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.config import Config


class DatasetProcessing(Dataset):
  def load_dataset(self, dataset_dir, is_train):
    # define hand class
    self.add_class("dataset", 1, "hand")
    # define data locations
    images_dir = dataset_dir + '/images/'
    boxes_dir = dataset_dir + '/boxes/'
    n = -1
    for filename in listdir(images_dir): #extract images
      n = n + 1
      image_id = filename[:-4]
      # we take 4 images out of 5 for the training
      if is_train and n%5 == 0:
        continue
      # we take one image every 5 for the test
      if not is_train and n%5 > 0:
        continue
      img_path = images_dir + filename
      box_path = boxes_dir + image_id + '.xml'
      # add to dataset
      self.add_image('dataset', image_id=image_id, path=img_path, box=box_path)
    

      # load the boxes for an image
  def load_mask(self, image_id):
    # get details of image
    info = self.image_info[image_id]
    # define box file location
    path = info['box']
    # load XML
    boxes, w, h = self.get_boxes(path)
    # create one array for all masks, each on a different channel
    arr_boxes = zeros([h, w, len(boxes)], dtype='uint8')
    # create masks
    class_ids = list()
    for i in range(len(boxes)):
      box = boxes[i]
      row_s, row_e = box[1], box[3]
      col_s, col_e = box[0], box[2]
      arr_boxes[row_s:row_e, col_s:col_e, i] = 1
      class_ids.append(self.class_names.index('hand'))
    return arr_boxes, np.asarray(class_ids, dtype='int32')
  
      # extract bounding boxes from xml file

  def get_boxes(self, filename):
    # load and parse the file
    tree = ElementTree.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = list()
    for box in root.findall('.//bndbox'):
      xmin = int(box.find('xmin').text)
      ymin = int(box.find('ymin').text)
      xmax = int(box.find('xmax').text)
      ymax = int(box.find('ymax').text)
      coors = [xmin, ymin, xmax, ymax]
      boxes.append(coors)
    # extract image dimensions
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height

  # load an image reference
  def image_reference(self, image_id):
    info = self.image_info[image_id]
    return info['path']



#Dataset structure:
#/dataset/
#        /images/
#        /boxes/

Split data into train and test set:

In [ ]:
from mrcnn.model import MaskRCNN
from matplotlib import pyplot
# train set
train_set = DatasetProcessing()
train_set.load_dataset('/content/drive/My Drive/Colab Notebooks', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# load an image
image_id = 1
image = train_set.load_image(image_id)
# load image mask
mask, class_ids = train_set.load_mask(image_id)

bbox = extract_bboxes(mask)
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

# plot mask
for i in range(mask.shape[2]):
  pyplot.imshow(mask[:, :, i], cmap='gray', alpha=0.2)
pyplot.show()

# test set
test_set = DatasetProcessing()
test_set.load_dataset('/content/drive/My Drive/Colab Notebooks', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Using TensorFlow backend.


Train: 4056
Test: 1015


Start training:

In [ ]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

# define a configuration for the model
class HandTraining(Config):
  NAME = "hand_training"
	# Number of classes(hand and no hand)
  NUM_CLASSES = 1 + 1
  # images_per_gpu * n_gpus = batch_size
  IMAGES_PER_GPU = 2
	# Number of training steps per epoch (ideally number of training samples/batch size)
  STEPS_PER_EPOCH = 50
# prepare config
config = HandTraining()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=2, layers='all')


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor










/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1987: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/2
49/50 [============================>.] - ETA: 3s - loss: 7.0138 - rpn_class_loss: 1.1278 - rpn_bbox_loss: 3.3564 - mrcnn_class_loss: 0.0887 - mrcnn_bbox_loss: 1.8438 - mrcnn_mask_loss: 0.5971

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2142: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'



50/50 [==============================] - 347s - loss: 6.9273 - rpn_class_loss: 1.1086 - rpn_bbox_loss: 3.3092 - mrcnn_class_loss: 0.0897 - mrcnn_bbox_loss: 1.8211 - mrcnn_mask_loss: 0.5986 - val_loss: 2.6507 - val_rpn_class_loss: 0.1359 - val_rpn_bbox_loss: 0.9378 - val_mrcnn_class_loss: 0.1326 - val_mrcnn_bbox_loss: 0.7842 - val_mrcnn_mask_loss: 0.6601
Epoch 2/2
50/50 [==============================] - 110s - loss: 2.5485 - rpn_class_loss: 0.1735 - rpn_bbox_loss: 0.9255 - mrcnn_class_loss: 0.1177 - mrcnn_bbox_loss: 0.7154 - mrcnn_mask_loss: 0.6164 - val_loss: 2.4555 - val_rpn_class_loss: 0.1781 - val_rpn_bbox_loss: 0.8478 - val_mrcnn_class_loss: 0.1138 - val_mrcnn_bbox_loss: 0.7116 - val_mrcnn_mask_loss: 0.6042


Make inference model and save it:

In [ ]:
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "hand_training"
	# number of classes (hand and no hand)
	NUM_CLASSES = 1 + 1

	GPU_COUNT = 1
	IMAGES_PER_GPU = 1 #1 because error otherwise

# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])

		APs.append(AP)
	# calculate the mean AP of all images
	mAP = mean(APs)
	return mAP


cfg = PredictionConfig()
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights
model.load_weights('/content/hand_training_2_epochs/weights_2_epochs.h5', by_name=True)
model.keras_model.save('/content/hand_training_2_epochs/weights_2_epochs.h5')



Instructions for updating:
Use `tf.cast` instead.
